In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./data/d_cll.csv")
data.head()

,preserves,work_order_number,site_number,site_type,date_created,month_year,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status,count_date
0,Frozen Food,12904684,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01,2012-01-02 10:00:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567391,HALF GLASS DOOR & WELL,In Service,2684
1,Frozen Food,12904685,A0892,SUPERMARKET,2012-01-02 03:05:00,2012-01,2012-01-02 09:59:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567398,FULL GLASS DOOR,In Service,2684
2,Frozen Food,12904853,A0892,SUPERMARKET,2012-01-02 05:10:00,2012-01,2012-01-02 10:01:00,Space Engineering services LTD,EOL,REFRIGERATION,...,Standard SLA,CARTER,2011-01-06 15:19:00,Region N10,DN31 1UF,42639.0,SSL00567399,FULL GLASS DOOR,In Service,2684
3,"Meat, Fish and Poultry",12927899,A0416,SUPERMARKET,2012-01-02 08:40:00,2012-01,2012-02-02 13:03:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,LINDE,2002-01-08 05:00:00,Region S01,BR4 0PU,13423.0,SSL00225160,REMOTE MULTIDECK,In Service,2684
4,Frozen Food,12932231,A0003,SUPERMARKET,2012-01-02 13:33:00,2012-01,2012-08-02 19:39:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,Region S02,DA11 8JH,37870.0,SSL00583503,FULL GLASS DOOR,Scrapped,2684


In [3]:
%%time
data["date_created"] = pd.DatetimeIndex(data.date_created)
data["date_completed"] = pd.DatetimeIndex(data.date_completed)
data["asset_in_service_date"] = pd.DatetimeIndex(data.asset_in_service_date)

CPU times: total: 531 ms
Wall time: 935 ms


In [4]:
data_prueba = data.date_created.dt.to_period('M').value_counts().reset_index()
data_prueba.columns = ["month_year","count_date"]
data_prueba = data_prueba.sort_values(by=['month_year'])
data_prueba.head()

,month_year,count_date
47,2012-01,2684
44,2012-02,4783
40,2012-03,6833
43,2012-04,5467
14,2012-05,12312


In [5]:
target = data["count_date"]
data_filtered = data[["count_date","site_type","date_created","date_completed","provider_name","asset_category","problem_type_name",
               "requested_by","wo_priority","manufacturer","asset_in_service_date","sales_sq_ft",
               "asset_type","asset_status","preserves"]]
data_filtered.head()

,count_date,site_type,date_created,date_completed,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,sales_sq_ft,asset_type,asset_status,preserves
0,2684,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 10:00:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,HALF GLASS DOOR & WELL,In Service,Frozen Food
1,2684,SUPERMARKET,2012-01-02 03:05:00,2012-01-02 09:59:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,FULL GLASS DOOR,In Service,Frozen Food
2,2684,SUPERMARKET,2012-01-02 05:10:00,2012-01-02 10:01:00,Space Engineering services LTD,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,CARTER,2011-01-06 15:19:00,42639.0,FULL GLASS DOOR,In Service,Frozen Food
3,2684,SUPERMARKET,2012-01-02 08:40:00,2012-02-02 13:03:00,CMS Solutions Limited,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,LINDE,2002-01-08 05:00:00,13423.0,REMOTE MULTIDECK,In Service,"Meat, Fish and Poultry"
4,2684,SUPERMARKET,2012-01-02 13:33:00,2012-08-02 19:39:00,CMS Solutions Limited,REFRIGERATION,Fixture Critical Over Temperature Alarm,Urgent Refrigeration Alarm,Standard SLA,RADFORD RETAIL SYSTEMS,2009-03-15 04:00:00,37870.0,FULL GLASS DOOR,Scrapped,Frozen Food


In [6]:
cat_columns = data_filtered.select_dtypes(['object']).columns
data_filtered[cat_columns] = data_filtered[cat_columns].apply(lambda x: pd.factorize(x)[0])
data_filtered.head()

,count_date,site_type,date_created,date_completed,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,asset_in_service_date,sales_sq_ft,asset_type,asset_status,preserves
0,2684,0,2012-01-02 03:05:00,2012-01-02 10:00:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,0,0,0
1,2684,0,2012-01-02 03:05:00,2012-01-02 09:59:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,1,0,0
2,2684,0,2012-01-02 05:10:00,2012-01-02 10:01:00,0,0,0,0,0,0,2011-01-06 15:19:00,42639.0,1,0,0
3,2684,0,2012-01-02 08:40:00,2012-02-02 13:03:00,1,0,0,0,0,1,2002-01-08 05:00:00,13423.0,2,0,1
4,2684,0,2012-01-02 13:33:00,2012-08-02 19:39:00,1,0,0,0,0,2,2009-03-15 04:00:00,37870.0,1,1,2


In [7]:
data_filtered["date_created_day"] = data_filtered["date_created"].apply(lambda x: x.day)
data_filtered["date_completed_day"] = data_filtered["date_completed"].apply(lambda x: x.day)
data_filtered["asset_in_service_date_day"] = data_filtered["asset_in_service_date"].apply(lambda x: x.day)

In [8]:
data_filtered["date_created_month"] = data_filtered["date_created"].apply(lambda x: x.month)
data_filtered["date_completed_month"] = data_filtered["date_completed"].apply(lambda x: x.month)
data_filtered["asset_in_service_date_month"] = data_filtered["asset_in_service_date"].apply(lambda x: x.month)

In [9]:
data_filtered["date_created_year"] = data_filtered["date_created"].apply(lambda x: x.year)
data_filtered["date_completed_year"] = data_filtered["date_completed"].apply(lambda x: x.year)
data_filtered["asset_in_service_date_year"] = data_filtered["asset_in_service_date"].apply(lambda x: x.year)

In [10]:
data_filtered["date_created_hour"] = data_filtered["date_created"].apply(lambda x: x.hour)
data_filtered["date_completed_hour"] = data_filtered["date_completed"].apply(lambda x: x.hour)
data_filtered["asset_in_service_date_hour"] = data_filtered["asset_in_service_date"].apply(lambda x: x.hour)

In [11]:
data_filtered.drop("date_created",axis=1,inplace=True)
data_filtered.drop("date_completed",axis=1,inplace=True)
data_filtered.drop("asset_in_service_date",axis=1,inplace=True)

In [12]:
data_filtered.head()

,count_date,site_type,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,sales_sq_ft,asset_type,...,asset_in_service_date_day,date_created_month,date_completed_month,asset_in_service_date_month,date_created_year,date_completed_year,asset_in_service_date_year,date_created_hour,date_completed_hour,asset_in_service_date_hour
0,2684,0,0,0,0,0,0,0,42639.0,0,...,6,1,1,1,2012,2012,2011,3,10,15
1,2684,0,0,0,0,0,0,0,42639.0,1,...,6,1,1,1,2012,2012,2011,3,9,15
2,2684,0,0,0,0,0,0,0,42639.0,1,...,6,1,1,1,2012,2012,2011,5,10,15
3,2684,0,1,0,0,0,0,1,13423.0,2,...,8,1,2,1,2012,2012,2002,8,13,5
4,2684,0,1,0,0,0,0,2,37870.0,1,...,15,1,8,3,2012,2012,2009,13,19,4


In [13]:
data_filtered.shape

(527644, 24)

# Analisis utilizando años (2012,2013,2014) para predecir (2015)
Todo el dataset sin los valores del 2015 son de training

In [14]:
data_3points = data_filtered[data_filtered.date_created_year != 2015]
data_3points.shape # alarmas que no son del 2015

(403928, 24)

In [15]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
X = data_3points[data_3points.columns.tolist()[1:]].to_numpy()
y = data_3points[["count_date"]].to_numpy()

In [17]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [18]:
model = linear_model.LinearRegression()
model.fit(X,y)

LinearRegression()

Los valores del dataset en 2015 son de testing

In [19]:
data2015 = data_filtered[data_filtered.date_created_year == 2015]

In [20]:
X_test = data2015[data2015.columns.tolist()[1:]].to_numpy()
y_test = data2015[["count_date"]].to_numpy()

In [21]:
X_test = scaler.transform(X_test) # necesario hacer el mismo scaler que en los datos de training

In [22]:
y_pred = model.predict(X_test)

In [23]:
y_pred = np.int32(y_pred)
y_t = np.reshape(y_test,len(y_test))
y_p = np.reshape(y_pred,len(y_pred))

In [24]:
dicte={"y_test":y_t,"y_pred":y_p}
pd.DataFrame(dicte)

,y_test,y_pred
0,13835,13512
1,13835,13958
2,13835,12521
3,13835,12550
4,13835,12239
...,...,...
123711,13835,14903
123712,13835,14184
123713,13835,14859
123714,13835,14698


In [25]:
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 33750225.83
Variance score: -3.97


## Eleccion aleatoria de site
Se trata de responder si una buena aproximacion al resultado del 2016 es realizar un promedio de cada año para cada site_number.

In [26]:
len(data.site_number.unique()) # numero de site unicos.

1577

In [27]:
df_sample = data.groupby(["site_number","month_year"]).sample() # muestras aleatorias de cada mes
site_number = df_sample.site_number.to_numpy()
df_sample.head()

,preserves,work_order_number,site_number,site_type,date_created,month_year,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status,count_date
2789,Frozen Food,13227270,A0002,SUPERMARKET,2012-01-03 00:04:00,2012-01,2012-03-03 12:06:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,MILLENNIUM MKII,2002-01-08 05:00:00,Region S04,E15 1XE,11846.0,SSL00601149,COLDROOM CONTROL PANEL,In Service,2684
2687,Frozen Food,13218793,A0002,SUPERMARKET,2012-02-29 02:35:00,2012-02,2012-02-29 05:49:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,MILLENNIUM MKII,2002-01-08 05:00:00,Region S04,E15 1XE,11846.0,SSL00601149,COLDROOM CONTROL PANEL,In Service,4783
9201,Frozen Food,13581721,A0002,SUPERMARKET,2012-03-31 08:50:00,2012-03,2012-01-04 11:04:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,MILLENNIUM MKII,2002-01-08 05:00:00,Region S04,E15 1XE,11846.0,SSL00601149,COLDROOM CONTROL PANEL,In Service,6833
11915,Frozen Food,13817109,A0002,SUPERMARKET,2012-04-20 00:55:00,2012-04,2012-04-25 14:27:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,MILLENNIUM MKII,2002-01-08 05:00:00,Region S04,E15 1XE,11846.0,SSL00601149,COLDROOM CONTROL PANEL,In Service,5467
20921,"Meat, Fish and Poultry",14222980,A0002,SUPERMARKET,2012-05-23 20:25:00,2012-05,2012-06-06 00:17:00,CMS Solutions Limited,EOL,REFRIGERATION,...,Standard SLA,MILLENNIUM MKII,2002-01-08 05:00:00,Region S04,E15 1XE,11846.0,SSL00601129,COLDROOM CONTROL PANEL,In Service,12312


In [28]:
cat_columns = df_sample.select_dtypes(['object']).columns
df_sample[cat_columns] = df_sample[cat_columns].apply(lambda x: pd.factorize(x)[0])
df_sample.head()

,preserves,work_order_number,site_number,site_type,date_created,month_year,date_completed,provider_name,wo_current_status,asset_category,...,wo_priority,manufacturer,asset_in_service_date,level_1_organization_name,site_zipcode,sales_sq_ft,asset_barcode,asset_type,asset_status,count_date
2789,0,13227270,0,0,2012-01-03 00:04:00,0,2012-03-03 12:06:00,0,0,0,...,0,0,2002-01-08 05:00:00,0,0,11846.0,0,0,0,2684
2687,0,13218793,0,0,2012-02-29 02:35:00,1,2012-02-29 05:49:00,0,0,0,...,0,0,2002-01-08 05:00:00,0,0,11846.0,0,0,0,4783
9201,0,13581721,0,0,2012-03-31 08:50:00,2,2012-01-04 11:04:00,0,0,0,...,0,0,2002-01-08 05:00:00,0,0,11846.0,0,0,0,6833
11915,0,13817109,0,0,2012-04-20 00:55:00,3,2012-04-25 14:27:00,0,0,0,...,0,0,2002-01-08 05:00:00,0,0,11846.0,0,0,0,5467
20921,1,14222980,0,0,2012-05-23 20:25:00,4,2012-06-06 00:17:00,0,0,0,...,0,0,2002-01-08 05:00:00,0,0,11846.0,1,0,0,12312


In [29]:
df_sample["date_created_day"] = df_sample["date_created"].apply(lambda x: x.day)
df_sample["date_completed_day"] = df_sample["date_completed"].apply(lambda x: x.day)
df_sample["asset_in_service_date_day"] = df_sample["asset_in_service_date"].apply(lambda x: x.day)

df_sample["date_created_month"] = df_sample["date_created"].apply(lambda x: x.month)
df_sample["date_completed_month"] = df_sample["date_completed"].apply(lambda x: x.month)
df_sample["asset_in_service_date_month"] = df_sample["asset_in_service_date"].apply(lambda x: x.month)

df_sample["date_created_year"] = df_sample["date_created"].apply(lambda x: x.year)
df_sample["date_completed_year"] = df_sample["date_completed"].apply(lambda x: x.year)
df_sample["asset_in_service_date_year"] = df_sample["asset_in_service_date"].apply(lambda x: x.year)

df_sample["date_created_hour"] = df_sample["date_created"].apply(lambda x: x.hour)
df_sample["date_completed_hour"] = df_sample["date_completed"].apply(lambda x: x.hour)
df_sample["asset_in_service_date_hour"] = df_sample["asset_in_service_date"].apply(lambda x: x.hour)

df_sample.drop("date_created",axis=1,inplace=True)
df_sample.drop("date_completed",axis=1,inplace=True)
df_sample.drop("asset_in_service_date",axis=1,inplace=True)

In [30]:
col = data2015[data2015.columns.tolist()[1:]].columns
df_sample_filtered = df_sample[col]
df_sample_filtered.head()

,site_type,provider_name,asset_category,problem_type_name,requested_by,wo_priority,manufacturer,sales_sq_ft,asset_type,asset_status,...,asset_in_service_date_day,date_created_month,date_completed_month,asset_in_service_date_month,date_created_year,date_completed_year,asset_in_service_date_year,date_created_hour,date_completed_hour,asset_in_service_date_hour
2789,0,0,0,0,0,0,0,11846.0,0,0,...,8,1,3,1,2012,2012,2002,0,12,5
2687,0,0,0,0,0,0,0,11846.0,0,0,...,8,2,2,1,2012,2012,2002,2,5,5
9201,0,0,0,0,0,0,0,11846.0,0,0,...,8,3,1,1,2012,2012,2002,8,11,5
11915,0,0,0,0,0,0,0,11846.0,0,0,...,8,4,4,1,2012,2012,2002,0,14,5
20921,0,0,0,0,0,0,0,11846.0,0,0,...,8,5,6,1,2012,2012,2002,20,0,5


In [31]:
month_sample = df_sample_filtered.date_created_month.to_numpy() # Mes para cada muestra seleccionada.

In [32]:
pred_2016_month = []
for i in range(len(df_sample_filtered)):
    prueba_2016 = df_sample_filtered.iloc[i].to_numpy()
    prueba_2016[17] = 2016.0
    prueba_2016[18] = 2016.0
    prueba_2016 = scaler.transform([prueba_2016])
    pred_2016_month.append(np.int32(model.predict(prueba_2016)))

In [33]:
count_sample = np.array(pred_2016_month).reshape(-1,len(pred_2016_month))[0]
sample_dict = {"site_number":site_number,"alarm_count":count_sample,"month":month_sample}

In [34]:
pre_alarms = pd.DataFrame(sample_dict) # numero de alarmas en el 2016 para cada tienda unica dad
pre_alarms.head(10)

,site_number,alarm_count,month
0,A0002,15739,1
1,A0002,19250,2
2,A0002,17771,3
3,A0002,18683,4
4,A0002,19263,5
5,A0002,20400,6
6,A0002,20854,7
7,A0002,20093,8
8,A0002,19247,9
9,A0002,20088,10


In [35]:
def analysis_2016(site,month_n):
    print("Analisis para {} durante el mes {}\n".format(site,month_n))
    stad = []
    for i in range(2,6):
        try:
            if month_n <= 9: 
                month_nn = "0{}".format(month_n)
            else: month_nn = "{}".format(month_n)
            s_temp = data[(data.site_number == site) & (data.month_year == "201{}-{}".format(i,month_nn))]["count_date"].unique()[0]
            print("Fecha: 201{}-{}".format(i,month_nn),", numero de alarmas:",s_temp)
            stad.append(s_temp)
        except:
            pass
    
    print("\nResultado prediccion en la fecha 2016-{}:".format(month_nn),pre_alarms[(pre_alarms.site_number == site) & (pre_alarms.month == int(month_nn))]["alarm_count"].values[0])
    print("Resultado de promedio estadistico entre de alarmas durante los 4 años de datos:", int(np.mean(stad)))

In [36]:
analysis_2016("A0002",1)

Analisis para A0002 durante el mes 1

Fecha: 2012-01 , numero de alarmas: 2684
Fecha: 2013-01 , numero de alarmas: 9684
Fecha: 2014-01 , numero de alarmas: 9822
Fecha: 2015-01 , numero de alarmas: 13835

Resultado prediccion en la fecha 2016-01: 15739
Resultado de promedio estadistico entre de alarmas durante los 4 años de datos: 9006


In [37]:
analysis_2016("A0002",7)

Analisis para A0002 durante el mes 7

Fecha: 2012-07 , numero de alarmas: 10984
Fecha: 2013-07 , numero de alarmas: 32014
Fecha: 2014-07 , numero de alarmas: 22115
Fecha: 2015-07 , numero de alarmas: 11656

Resultado prediccion en la fecha 2016-07: 20854
Resultado de promedio estadistico entre de alarmas durante los 4 años de datos: 19192


In [38]:
analysis_2016("A0003",9)

Analisis para A0003 durante el mes 9

Fecha: 2012-09 , numero de alarmas: 6814
Fecha: 2013-09 , numero de alarmas: 13882
Fecha: 2014-09 , numero de alarmas: 13488
Fecha: 2015-09 , numero de alarmas: 8639

Resultado prediccion en la fecha 2016-09: 19946
Resultado de promedio estadistico entre de alarmas durante los 4 años de datos: 10705


In [39]:
analysis_2016("A0008",5)

Analisis para A0008 durante el mes 5

Fecha: 2012-05 , numero de alarmas: 12312
Fecha: 2013-05 , numero de alarmas: 10236
Fecha: 2014-05 , numero de alarmas: 12494
Fecha: 2015-05 , numero de alarmas: 10403

Resultado prediccion en la fecha 2016-05: 20586
Resultado de promedio estadistico entre de alarmas durante los 4 años de datos: 11361


In [40]:
analysis_2016("P2308",1)

Analisis para P2308 durante el mes 1

Fecha: 2015-01 , numero de alarmas: 13835

Resultado prediccion en la fecha 2016-01: 16794
Resultado de promedio estadistico entre de alarmas durante los 4 años de datos: 13835
